In [6]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()

file_path = filedialog.askopenfilename(filetypes=[("iCalendar files", "*.ics")])

print("Selected file:", file_path)
root.destroy()

Selected file: /home/tnmon/git/ical-to-csv/tiana@techandciviclife.org 3.ics


In [7]:
import tkinter as tk
from tkinter import ttk
import datetime

root = tk.Tk()


def get_month_increment():
    month = month_var.get()
    year = int(year_var.get())
    increment = int(increment_entry.get())
    global start_date 
    start_date = datetime.datetime(year, months.index(month) + 1, 1)
    global end_date 
    end_date = start_date + datetime.timedelta(days=31)
    end_date = datetime.datetime(end_date.year, end_date.month, 1) - datetime.timedelta(days=1)
    if increment > 1:
        end_date = start_date + datetime.timedelta(days=31 * increment)
        end_date = datetime.datetime(end_date.year, end_date.month, 1) - datetime.timedelta(days=1)
    print("Start date:", start_date.strftime("%Y-%m-%d"))
    # print("End date:", end_date.strftime("%Y-%m-%d"))
    root.destroy()

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
current_month = datetime.datetime.now().month
month_var = tk.StringVar(value=months[current_month - 2])

current_year = datetime.datetime.now().year
years = [str(year) for year in range(current_year, current_year - 7, -1)]
year_var = tk.StringVar(value=years[0])

month_label = tk.Label(root, text="Select a month:")
month_label.pack()

month_dropdown = ttk.Combobox(root, textvariable=month_var, values=months)
month_dropdown.pack()

year_label = tk.Label(root, text="Select a year:")
year_label.pack()

year_dropdown = ttk.Combobox(root, textvariable=year_var, values=years)
year_dropdown.pack()

increment_label = tk.Label(root, text="Enter the number of months to increment:")
increment_label.pack()

increment_entry = tk.Entry(root)
increment_entry.insert(0, "1")
increment_entry.pack()

button = tk.Button(root, text="OK", command=get_month_increment)
button.pack()

root.mainloop()

Start date: 2022-06-01


In [9]:
import csv
import datetime
import os
import webbrowser
import sys
import re
import tkinter as tk
from tkinter import filedialog

class App:
    def __init__(self, file_path, start_date, end_date):
        self.file_path = file_path
        self.start_date = start_date
        self.end_date = end_date
        self.root = tk.Tk()
        self.root.title("Generate Report")
        self.file_label = tk.Label(self.root, text=f"File path: {self.file_path}")
        self.file_label.pack()
        self.start_label = tk.Label(self.root, text=f"Start date: {self.start_date}")
        self.start_label.pack()
        self.end_label = tk.Label(self.root, text=f"End date: {self.end_date}")
        self.end_label.pack()
        self.generate_button = tk.Button(self.root, text="Generate", command=self.generate_report)
        self.generate_button.pack()

    def generate_report(self):
        events = self.extract_events(self.file_path, self.start_date, self.end_date)
        output_file_path = os.path.splitext(self.file_path)[0] + ".csv"
        self.write_csv(events, output_file_path)

    def extract_events(self, file_path, start_date, end_date):
        events = []
        with open(file_path, "r") as f:
            data = f.read()
        pattern = r"BEGIN:VEVENT.*?END:VEVENT"
        matches = re.findall(pattern, data, re.DOTALL)
        for match in matches:
            event = {}
            keys = ["DTSTART:", "DTEND:", "DTSTAMP:", "ORGANIZER;", "UID:", "ATTENDEE;", "CREATED:", "DESCRIPTION:", "LAST-MODIFIED:", "SEQUENCE:", "STATUS:", "SUMMARY:", "TRANSP:"]
            split_matches = re.split(r"(DTSTART:|DTEND:|DTSTAMP:|ORGANIZER;|UID:|ATTENDEE;|CREATED:|DESCRIPTION:|LAST-MODIFIED:|SEQUENCE:|STATUS:|SUMMARY:|TRANSP:)", match)
            for i in range(len(split_matches)):
                if split_matches[i] in keys:
                    key = split_matches[i]
                    value = split_matches[i+1]
                    if key == "SUMMARY:":
                        event["title"] = value
                    elif key == "DTSTART:":
                        start_time_str = value
                        event["start time"] = datetime.datetime.strptime(start_time_str, "%Y%m%dT%H%M%SZ\n")
                    elif key == "DTEND:":
                        end_time_str = re.sub(r";BYDAY=(\w\w)", "", value)
                        try: event["end time"] = datetime.datetime.strptime(end_time_str, "%Y%m%dT%H%M%SZ\n")
                        except ValueError: event["end time"]= 'unknown'
                    elif key == "ORGANIZER;":
                        try: organizer_name = re.search(r"=(.*?):", value).group(1)
                        except AttributeError: organizer_name = 'unknown'
                        event["organizer"] = organizer_name
                    elif key == "ATTENDEE;":
                        if "attendees" not in event:
                            event["attendees"] = []
                        try: attendee_name = re.search(r"CN=([^;]+)", value).group(1)
                        except AttributeError: attendee_name = ''
                        try: attendee_status = re.search(r"PARTSTAT=([^;]+)", value).group(1)
                        except AttributeError: attendee_name = ''
                        attendee_name = attendee_name.replace('\n ', '')
                        if attendee_status == "DECLINED":
                            attendee_name = f"({attendee_name})**"
                        if attendee_status == "NEEDS-ACTION":
                            attendee_name = f"{attendee_name}(?)"
                        event["attendees"].append(attendee_name)

            if "start time" in event and "end time" in event and start_date <= event["start time"] <= end_date:
                try: duration = (event["end time"] - event["start time"]).total_seconds() / 3600 or 'unknown'
                except TypeError: duration = "unknown"
                event["duration"] = duration
                events.append(event)
        return events

    def write_csv(self, events, file_path):
        with open(file_path, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["title", "start time", "end time", "duration", "organizer", "attendees"])
            writer.writeheader()
            for event in events:
                try:
                    attendees = sorted(event["attendees"])
                    event["attendees"] = ", ".join(attendees)
                except KeyError: event["attendees"] = 'n/a'
                event["start time"] = event["start time"].strftime("%Y-%m-%d")
                try: event["end time"] = event["end time"].strftime("%Y-%m-%d")
                except AttributeError: event["end time"] = "unknown"
                writer.writerow(event)
        try: webbrowser.open(os.path.dirname(file_path))
        except: sys.exit()

    def run(self):
        self.root.mainloop()


app = App(file_path, start_date, end_date)
app.run()


** (org.gnome.Nautilus:17729): WARNING **: 16:01:20.614: Unable to get contents of the bookmarks file: Error opening file /home/tnmon/.gtk-bookmarks: No such file or directory

** (org.gnome.Nautilus:17729): WARNING **: 16:01:20.615: Unable to get contents of the bookmarks file: Error opening file /home/tnmon/.gtk-bookmarks: No such file or directory
